##### Copyright 2020 Los autores de TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introducción a los gradientes y la diferenciación automática

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a></td>
</table>

## Diferenciación y gradientes automáticos

[La diferenciación automática](https://en.wikipedia.org/wiki/Automatic_differentiation) es útil para implementar algoritmos de aprendizaje automático como la [propagación](https://en.wikipedia.org/wiki/Backpropagation) hacia atrás para entrenar redes neuronales.

En esta guía, analizaremos las formas en que puede calcular gradientes con TensorFlow, especialmente en una ejecución ávida.

## Configuración

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

## Gradientes de computación

Para diferenciar automáticamente, TensorFlow necesita recordar qué operaciones suceden en qué orden durante el pase *hacia adelante.* Luego, durante la *pasada hacia atrás* , TensorFlow recorre esta lista de operaciones en orden inverso para calcular gradientes.

## Cintas de degradado

TensorFlow proporciona la [API tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) para la diferenciación automática; es decir, calcular el gradiente de un cálculo con respecto a algunas entradas, normalmente `tf.Variable` s. TensorFlow "registra" las operaciones relevantes ejecutadas dentro del contexto de un `tf.GradientTape` en una "cinta". Luego, TensorFlow usa esa cinta para calcular los gradientes de un cálculo "grabado" mediante [la diferenciación en modo inverso](https://en.wikipedia.org/wiki/Automatic_differentiation) .

A continuación, se muestra un ejemplo sencillo:

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2

Una vez que haya registrado algunas operaciones, use `GradientTape.gradient(target, sources)` para calcular el gradiente de algún destino (a menudo una pérdida) en relación con alguna fuente (a menudo las variables del modelo).

In [ ]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

El ejemplo anterior usa escalares, pero `tf.GradientTape` funciona con la misma facilidad en cualquier tensor:

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

Para obtener el gradiente de `y` con respecto a ambas variables, puede pasar ambas como fuentes al método de `gradient` La cinta es flexible sobre cómo se pasan las fuentes y aceptará cualquier combinación anidada de listas o diccionarios y devolverá el degradado estructurado de la misma manera (ver `tf.nest` ).

In [ ]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

El gradiente con respecto a cada fuente tiene la forma de la fuente:

In [ ]:
print(w.shape)
print(dl_dw.shape)

Aquí está el cálculo del gradiente nuevamente, esta vez pasando un diccionario de variables:

In [ ]:
my_vars = {
    'w': tf.Variable(tf.random.normal((3, 2)), name='w'),
    'b': tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
}

grad = tape.gradient(loss, my_vars)
grad['b']

## Degradados con respecto a un modelo

Es común recopilar `tf.Variables` en un `tf.Module` o una de sus subclases ( `layers.Layer` , `keras.Model` ) para realizar [puntos](checkpoint.ipynb) de control y [exportar](saved_model.ipynb) .

En la mayoría de los casos, querrá calcular gradientes con respecto a las variables entrenables de un modelo. Dado que todas las subclases de `tf.Module` agregan sus variables en la `Module.trainable_variables` , puede calcular estos gradientes en unas pocas líneas de código: 

In [ ]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)

In [ ]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape: {g.shape}')

<a id="watches"></a>

## Controlando lo que mira la cinta

El comportamiento predeterminado es registrar todas las operaciones después de acceder a un `tf.Variable` entrenable. Las razones de esto son:

- La cinta necesita saber qué operaciones grabar en la pasada hacia adelante para calcular los gradientes en la pasada hacia atrás.
- La cinta contiene referencias a salidas intermedias, por lo que no desea grabar operaciones innecesarias.
- El caso de uso más común implica calcular el gradiente de una pérdida con respecto a todas las variables entrenables de un modelo.

Por ejemplo, lo siguiente no puede calcular un gradiente porque `tf.Tensor` no se "ve" de forma predeterminada y `tf.Variable` no se puede entrenar:

In [ ]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

Puede enumerar las variables que observa la cinta mediante el método `GradientTape.watched_variables`

In [ ]:
[var.name for var in tape.watched_variables()]

`tf.GradientTape` proporciona ganchos que le dan al usuario control sobre lo que se ve o no.

Para registrar gradientes con respecto a un `tf.Tensor` , debe llamar a `GradientTape.watch(x)` :

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x**2

# dy = 2x * dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

Por el contrario, para deshabilitar el comportamiento predeterminado de ver todas las `tf.Variables` , configure `watch_accessed_variables=False` al crear la cinta de degradado. Este cálculo usa dos variables, pero solo conecta el gradiente para una de las variables:

In [ ]:
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

Dado que no se llamó a `GradientTape.watch` `x0` , no se calcula ningún gradiente con respecto a él:

In [ ]:
# dy = 2x * dx
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print('dy/dx0:', grad['x0'])
print('dy/dx1:', grad['x1'].numpy())

## Resultados intermedios

También puede solicitar gradientes de la salida con respecto a los valores intermedios calculados dentro del contexto `tf.GradientTape`

In [ ]:
x = tf.constant(3.0)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = x * x
  z = y * y

# Use the tape to compute the gradient of z with respect to the
# intermediate value y.
# dz_dx = 2 * y, where y = x ** 2
print(tape.gradient(z, y).numpy())

De forma predeterminada, los recursos de un `GradientTape` se liberan tan pronto como se llama al método `GradientTape.gradient()` Para calcular varios degradados sobre el mismo cálculo, cree una cinta de degradado `persistent` Esto permite múltiples llamadas al `gradient()` medida que se liberan recursos cuando el objeto de cinta se recolecta como basura. Por ejemplo:

In [ ]:
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy())  # 108.0 (4 * x**3 at x = 3)
print(tape.gradient(y, x).numpy())  # 6.0 (2 * x)

In [ ]:
del tape   # Drop the reference to the tape

## Notas sobre el rendimiento

- Hay una pequeña sobrecarga asociada con la realización de operaciones dentro de un contexto de cinta de degradado. Para la ejecución más ávida, esto no será un costo notable, pero aún debe usar el contexto de cinta alrededor de las áreas solo donde sea necesario.

- Las cintas de degradado utilizan la memoria para almacenar resultados intermedios, incluidas las entradas y salidas, para su uso durante la pasada hacia atrás.

    Para mayor eficiencia, algunas operaciones (como `ReLU` ) no necesitan mantener sus resultados intermedios y se podan durante el pase hacia adelante. Sin embargo, si usa `persistent=True` en su cinta, *no se descarta nada* y su uso máximo de memoria será mayor.

## Gradientes de objetivos no escalares

Un gradiente es fundamentalmente una operación sobre un escalar.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient(y0, x).numpy())
print(tape.gradient(y1, x).numpy())

Por lo tanto, si solicita el gradiente de varios objetivos, el resultado para cada fuente es:

- El gradiente de la suma de los objetivos, o equivalentemente
- La suma de los gradientes de cada objetivo.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient({'y0': y0, 'y1': y1}, x).numpy())

Del mismo modo, si los objetivos no son escalares, se calcula el gradiente de la suma:

In [ ]:
x = tf.Variable(2.)

with tf.GradientTape() as tape:
  y = x * [3., 4.]

print(tape.gradient(y, x).numpy())

Esto hace que sea sencillo tomar el gradiente de la suma de una colección de pérdidas, o el gradiente de la suma de un cálculo de pérdidas por elementos.

Si necesita un gradiente separado para cada elemento, consulte [Jacobians](advanced_autodiff.ipynb#jacobians) .

En algunos casos, puede omitir el jacobiano. Para un cálculo por elementos, el gradiente de la suma da la derivada de cada elemento con respecto a su elemento de entrada, ya que cada elemento es independiente:

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.nn.sigmoid(x)

dy_dx = tape.gradient(y, x)

In [ ]:
plt.plot(x, y, label='y')
plt.plot(x, dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

## Flujo de control

Debido a que las cintas registran las operaciones a medida que se ejecutan, el flujo de control de Python (usando `if` sy `while` s, por ejemplo) se maneja naturalmente.

Aquí se usa una variable diferente en cada rama de un `if` . El gradiente solo se conecta a la variable que se utilizó:

In [ ]:
x = tf.constant(1.0)

v0 = tf.Variable(2.0)
v1 = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  if x > 0.0:
    result = v0
  else:
    result = v1**2 

dv0, dv1 = tape.gradient(result, [v0, v1])

print(dv0)
print(dv1)

Solo recuerde que las declaraciones de control en sí mismas no son diferenciables, por lo que son invisibles para los optimizadores basados en gradientes.

Dependiendo del valor de `x` en el ejemplo anterior, la cinta registra `result = v0` o `result = v1**2` . El gradiente con respecto `x` siempre es `None` .

In [ ]:
dx = tape.gradient(result, x)

print(dx)

## Obteniendo un gradiente de `None`

Cuando un objetivo no está conectado a una fuente, obtendrá un gradiente de `None` .


In [ ]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y * y
print(tape.gradient(z, x))

Aquí, `z` no está conectado `x` , pero hay varias formas menos obvias de desconectar un gradiente.

### 1. Reemplazó una variable con un tensor.

En la sección sobre ["controlar lo que mira la cinta"](#watches) , vio que la cinta automáticamente verá una `tf.Variable` pero no un `tf.Tensor` .

Un error común es reemplazar inadvertidamente un `tf.Variable` con un `tf.Tensor` , en lugar de usar `Variable.assign` para actualizar `tf.Variable` . Aquí hay un ejemplo:

In [ ]:
x = tf.Variable(2.0)

for epoch in range(2):
  with tf.GradientTape() as tape:
    y = x+1

  print(type(x).__name__, ":", tape.gradient(y, x))
  x = x + 1   # This should be `x.assign_add(1)`

### 2. Hizo cálculos fuera de TensorFlow

La cinta no puede registrar la ruta del gradiente si el cálculo sale de TensorFlow. Por ejemplo:

In [ ]:
x = tf.Variable([[1.0, 2.0],
                 [3.0, 4.0]], dtype=tf.float32)

with tf.GradientTape() as tape:
  x2 = x**2

  # This step is calculated with NumPy
  y = np.mean(x2, axis=0)

  # Like most ops, reduce_mean will cast the NumPy array to a constant tensor
  # using `tf.convert_to_tensor`.
  y = tf.reduce_mean(y, axis=0)

print(tape.gradient(y, x))

### 3. Tomó degradados a través de un número entero o una cadena.

Los enteros y las cadenas no son diferenciables. Si una ruta de cálculo utiliza estos tipos de datos, no habrá gradiente.

Nadie espera que las cadenas sean diferenciables, pero es fácil crear accidentalmente una `int` si no especificas el tipo `dtype` .

In [ ]:
# The x0 variable has an `int` dtype.
x = tf.Variable([[2, 2],
                 [2, 2]])

with tf.GradientTape() as tape:
  # The path to x1 is blocked by the `int` dtype here.
  y = tf.cast(x, tf.float32)
  y = tf.reduce_sum(x)

print(tape.gradient(y, x))

TensorFlow no transmite automáticamente entre tipos, por lo que en la práctica a menudo obtendrá un error de tipo en lugar de un degradado faltante.

### 5. Tomó degradados a través de un objeto con estado.

El estado detiene los gradientes. Cuando lee de un objeto con estado, la cinta solo puede ver el estado actual, no el historial que conduce a él.

Un `tf.Tensor` es inmutable. No puedes cambiar un tensor una vez creado. Tiene un *valor* , pero no un *estado* . Todas las operaciones discutidas hasta ahora también son apátridas: la salida de un `tf.matmul` solo depende de sus entradas.

Una `tf.Variable` tiene un estado interno, su valor. Cuando usa la variable, se lee el estado. Es normal calcular un gradiente con respecto a una variable, pero el estado de la variable bloquea los cálculos de gradiente para que no retrocedan más. Por ejemplo:


In [ ]:
x0 = tf.Variable(3.0)
x1 = tf.Variable(0.0)

with tf.GradientTape() as tape:
  # Update x1 = x1 + x0.
  x1.assign_add(x0)
  # The tape starts recording from x1.
  y = x1**2   # y = (x1 + x0)**2

# This doesn't work.
print(tape.gradient(y, x0))   #dy/dx0 = 2*(x1 + x2)

`tf.data.Dataset` similar, los iteradores `tf.queue` s tienen estado y detendrán todos los gradientes en los tensores que los atraviesen.

## Sin gradiente registrado

Algunas `tf.Operation` se **registran como no diferenciables** y devolverán `None` . Otros **no tienen ningún gradiente registrado** .

La [página tf.raw_ops](https://www.tensorflow.org/api_docs/python/tf/raw_ops) muestra qué operaciones de bajo nivel tienen gradientes registrados.

Si intenta tomar un gradiente a través de una operación flotante que no tiene gradiente registrado, la cinta arrojará un error en lugar de devolver silenciosamente `None` . De esta manera sabrá que algo salió mal.

Por ejemplo, la función `tf.image.adjust_contrast` [envuelve raw_ops.AdjustContrastv2](https://www.tensorflow.org/api_docs/python/tf/raw_ops#.AdjustContrastv2) que podría tener un degradado pero el degradado no está implementado:


In [ ]:
image = tf.Variable([[[0.5, 0.0, 0.0]]])
delta = tf.Variable(0.1)

with tf.GradientTape() as tape:
  new_image = tf.image.adjust_contrast(image, delta)

try:
  print(tape.gradient(new_image, [image, delta]))
  assert False   # This should not happen.
except LookupError as e:
  print(f'{type(e).__name__}: {e}')


`tf.RegisterGradient` , deberá implementar el gradiente y registrarlo (usando tf.RegisterGradient), o volver a implementar la función usando otras operaciones.

## Ceros en lugar de Ninguno

En algunos casos, sería conveniente obtener 0 en lugar de `None` para gradientes no conectados. Puede decidir qué devolver cuando tiene gradientes no conectados utilizando el argumento `unconnected_gradients`

In [ ]:
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))